<a href="https://colab.research.google.com/github/Ace1811/Data-Science---Standard-Projects/blob/master/Standard_Project_4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tsfresh

Да се изберат пет различни временски серии кои имаат во себе податоци за изминатите 4 години (на пример податоци од берза, криптовалути, цена на нафта, цена на злато, цена на земен гас). Потоа, да се проучат податоците со користење на tsfresh и истите да се објаснат. За крај, да се направат различни интерактивни визуелизации на сите податоци кои ќе овозможат лесна компаративна анализа (да се направи истото со најмалку две библиотеки и да се објасни која е подобра).

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tsfresh import extract_features
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px

# Step 1: Collect Data
# Collect time series data for different financial instruments, commodities, etc. spanning the past 4 years.
# Store the data in pandas DataFrames.

btc = pd.read_csv('') #додади множества
eth = pd.read_csv('') #додади множества
tether_usdt = pd.read_csv('') #додади множества
bnb = pd.read_csv('')   #додади множества
xrp = pd.read_csv('')   #додади множества

btc=btc.add_prefix('btc')
eth=eth.add_prefix('eth')
tether_usdt=tether_usdt.add_prefix('tether_usdt')
bnb=bnb.add_prefix('bnb')
xrp=xrp.add_prefix('xrp')

btc.rename(columns={'btcDate': "Date"}, inplace=True)
eth.rename(columns={'ethDate': "Date"}, inplace=True)
tether_usdt.rename(columns={'tether_usdtDate': "Date"}, inplace=True)
bnb.rename(columns={'bnbDate': "Date"}, inplace=True)
xrp.rename(columns={'xrpDate': "Date"}, inplace=True)


btc



,Date,btcOpen,btcHigh,btcLow,btcClose,btcAdj Close,btcVolume
0,2018-08-26,6754.640137,6774.750000,6620.750000,6707.259766,6707.259766,3295500000
1,2018-08-27,6710.799805,6884.640137,6689.709961,6884.640137,6884.640137,4019000000
2,2018-08-28,6891.080078,7109.560059,6882.339844,7096.279785,7096.279785,4659940000
3,2018-08-29,7091.709961,7113.299805,6970.819824,7047.160156,7047.160156,4145880000
4,2018-08-30,7043.759766,7072.689941,6834.689941,6978.229980,6978.229980,4463250000
...,...,...,...,...,...,...,...
1822,2023-08-22,26130.748047,26135.507813,25520.728516,26031.656250,26031.656250,14503820706
1823,2023-08-23,26040.474609,26786.898438,25804.998047,26431.640625,26431.640625,16985265785
1824,2023-08-24,26431.519531,26554.910156,25914.925781,26162.373047,26162.373047,12871532023
1825,2023-08-25,26163.679688,26248.103516,25786.812500,26047.667969,26047.667969,12406045118


In [ ]:
# Step 2: Data Preparation
# Prepare the collected time series data for analysis.
# Ensure data is in the appropriate format (date, value columns).
# Convert data types, handle missing values, etc.

# Merge the first two DataFrames
merged_df = pd.merge(btc, eth, on='Date', how='outer')

# Merge the rest of the DataFrames
datasets = [tether_usdt, bnb, xrp]
for dataset in datasets:
    merged_df = pd.merge(merged_df, dataset, on='Date', how='outer')


merged_df



columns_to_drop = [col for col in merged_df.columns if 'Open' not in col and col != 'Date'] #funkcija za dropnuvanje na site osven OPEN & Date

# Drop the specified columns
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df #gi mergnuvame

print(type(merged_df['Date'][0]))
# Convert the 'date' string column to datetime data type
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
print(type(merged_df['Date'][0]))
merged_df

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,Date,btcOpen,ethOpen,tether_usdtOpen,bnbOpen,xrpOpen
0,2018-08-26,6754.640137,279.524994,1.003210,10.114200,0.328162
1,2018-08-27,6710.799805,275.350006,1.001640,10.219000,0.323643
2,2018-08-28,6891.080078,286.648987,0.997670,11.160700,0.337527
3,2018-08-29,7091.709961,296.162994,1.000190,11.471000,0.350833
4,2018-08-30,7043.759766,289.752991,0.999735,11.169200,0.344546
...,...,...,...,...,...,...
1822,2023-08-22,26130.748047,1667.279297,0.999796,210.579956,0.524536
1823,2023-08-23,26040.474609,1634.401733,0.999613,210.818695,0.520919
1824,2023-08-24,26431.519531,1679.248291,0.999829,216.444839,0.529913
1825,2023-08-25,26163.679688,1660.269897,0.999484,218.681091,0.518458


In [ ]:
melted_df = pd.melt(merged_df, id_vars=['Date'], var_name='Value', value_name='Price') #funkcija melt za da vleze datasetot vo tsfresh - extract features

print(melted_df)

           Date    Value        Price
0    2018-08-26  btcOpen  6754.640137
1    2018-08-27  btcOpen  6710.799805
2    2018-08-28  btcOpen  6891.080078
3    2018-08-29  btcOpen  7091.709961
4    2018-08-30  btcOpen  7043.759766
...         ...      ...          ...
9130 2023-08-22  xrpOpen     0.524536
9131 2023-08-23  xrpOpen     0.520919
9132 2023-08-24  xrpOpen     0.529913
9133 2023-08-25  xrpOpen     0.518458
9134 2023-08-26  xrpOpen     0.526259

[9135 rows x 3 columns]


In [ ]:
# Step 3: Feature Extraction with tsfresh
# Extract relevant features from the time series data using tsfresh.
# This step transforms raw time series data into a format suitable for analysis.


# For example:
extracted_features = extract_features(melted_df, column_id='Value', column_sort='Date') #so tsfresh vadime nekoj podatoci/statistiki i tie se vo kolonite krsteni

Feature Extraction: 100%|██████████| 5/5 [00:20<00:00,  4.06s/it]


In [ ]:
extracted_features

,Price__variance_larger_than_standard_deviation,Price__has_duplicate_max,Price__has_duplicate_min,Price__has_duplicate,Price__sum_values,Price__abs_energy,Price__mean_abs_change,Price__mean_change,Price__mean_second_derivative_central,Price__median,...,Price__fourier_entropy__bins_5,Price__fourier_entropy__bins_10,Price__fourier_entropy__bins_100,Price__permutation_entropy__dimension_3__tau_1,Price__permutation_entropy__dimension_4__tau_1,Price__permutation_entropy__dimension_5__tau_1,Price__permutation_entropy__dimension_6__tau_1,Price__permutation_entropy__dimension_7__tau_1,Price__query_similarity_count__query_None__threshold_0.0,Price__mean_n_absolute_max__number_of_maxima_7
bnbOpen,1.0,0.0,0.0,0.0,3.404180e+05,1.229698e+08,5.997735,0.114100,-8.907014e-05,210.818695,...,0.090729,0.170467,0.459635,1.760757,3.041288,4.442097,5.798957,6.748892,NaN,659.429313
btcOpen,1.0,0.0,0.0,0.0,4.134980e+07,1.431095e+12,566.333775,10.565263,-2.000789e-02,19139.000000,...,0.090729,0.136002,0.414868,1.752627,3.023217,4.416004,5.769299,6.717974,NaN,65760.690848
ethOpen,1.0,0.0,0.0,1.0,2.357851e+06,5.656964e+09,41.920204,0.752088,-8.923671e-04,1204.555298,...,0.170467,0.170467,0.493589,1.757567,3.027484,4.432576,5.798658,6.770715,NaN,4692.233887
tether_usdtOpen,0.0,0.0,0.0,1.0,1.829222e+03,1.831489e+03,0.001569,-0.000002,4.268493e-07,1.000365,...,0.170467,0.170467,1.261378,1.784633,3.143958,4.691488,6.246824,7.197703,NaN,1.027716
xrpOpen,0.0,0.0,0.0,1.0,8.785738e+02,5.722076e+02,0.018452,0.000108,3.375342e-06,0.384697,...,0.170467,0.215617,0.656975,1.765325,3.051815,4.468748,5.868508,6.836906,NaN,1.694384


In [ ]:
means = extracted_features.filter(like='mean', axis=1) #filter da gi najdam site statistiki povrzani so mean (sredna vrednost)
median = extracted_features.filter(like='median', axis=1)
mode = extracted_features.filter(like='mode', axis=1)
change_quantiles = extracted_features.filter(like='change_quantiles', axis=1)
autocorrelation = extracted_features.filter(like='autocorrelation', axis=1)
approximate_entropy = extracted_features.filter(like='approximate_entropy', axis=1)

means.name = 'means'
median.name='median'
mode.name ='mode'
change_quantiles.name='change_quantiles'
autocorrelation.name='autocorrelation'
approximate_entropy.name='approximate_entropy'

datasets_extracted=[means,median,mode,change_quantiles,autocorrelation,approximate_entropy]




# Drop all columns except the ones in 'columns_to_keep'
##statistical_measures = extracted_features[columns_to_keep]
#statistical_measures

In [ ]:
means

,Price__mean_abs_change,Price__mean_change,Price__mean_second_derivative_central,Price__mean,Price__root_mean_square,Price__longest_strike_below_mean,Price__longest_strike_above_mean,Price__count_above_mean,Price__count_below_mean,"Price__agg_autocorrelation__f_agg_""mean""__maxlag_40",...,"Price__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","Price__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","Price__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""","Price__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","Price__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""","Price__agg_linear_trend__attr_""slope""__chunk_len_50__f_agg_""mean""","Price__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""","Price__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","Price__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""mean""",Price__mean_n_absolute_max__number_of_maxima_7
bnbOpen,5.997735,0.114100,-8.907014e-05,186.326200,259.435832,908.0,919.0,919.0,908.0,0.953422,...,-38.004859,-37.384690,-28.862381,1.229501,2.458887,11.986986,0.061842,0.174782,1.934973,659.429313
btcOpen,566.333775,10.565263,-2.000789e-02,22632.621722,27987.554859,845.0,543.0,771.0,1056.0,0.962443,...,6410.210033,6454.360676,7019.554553,88.921121,177.846579,871.273821,6.684663,18.910241,209.141892,65760.690848
ethOpen,41.920204,0.752088,-8.923671e-04,1290.559035,1759.634306,878.0,501.0,849.0,978.0,0.962945,...,-48.764890,-45.135521,1.442405,7.342049,14.684691,71.957862,0.449810,1.271717,13.950381,4692.233887
tether_usdtOpen,0.001569,-0.000002,4.268493e-07,1.001216,1.001228,436.0,57.0,548.0,1279.0,0.376598,...,1.002977,1.002973,1.002937,-0.000010,-0.000019,-0.000097,0.000002,0.000005,0.000055,1.027716
xrpOpen,0.018452,0.000108,3.375342e-06,0.480883,0.559638,735.0,417.0,603.0,1224.0,0.846093,...,0.315143,0.315590,0.320241,0.000909,0.001817,0.009009,0.000133,0.000374,0.003990,1.694384


In [ ]:
median

,Price__median,"Price__agg_autocorrelation__f_agg_""median""__maxlag_40"
bnbOpen,210.818695,0.954541
btcOpen,19139.000000,0.963736
ethOpen,1204.555298,0.963896
tether_usdtOpen,1.000365,0.373715
xrpOpen,0.384697,0.849140


In [ ]:
mode

""
bnbOpen
btcOpen
ethOpen
tether_usdtOpen
xrpOpen


In [ ]:
change_quantiles

,"Price__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0","Price__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","Price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","Price__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","Price__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0","Price__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","Price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","Price__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.0","Price__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","Price__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0",...,"Price__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6","Price__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.6","Price__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Price__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6","Price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6","Price__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.6","Price__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8","Price__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8","Price__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","Price__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.8"
bnbOpen,0.014478,0.351195,0.341728,0.234627,0.030226,0.593295,0.493011,0.351149,0.134416,16.463948,...,5.955496,4.121971e+01,-0.154220,3.375646e+02,11.655575,2.017360e+02,0.189309,5.565335e+02,16.491417,2.846025e+02
btcOpen,11.165000,34529.212458,114.480507,21548.083211,9.236184,69672.688135,159.250509,44397.270729,13.997065,163687.984831,...,685.192462,5.859683e+05,5.048997,2.352481e+06,1051.561324,1.246725e+06,7.225485,3.162892e+06,1290.217493,1.498283e+06
ethOpen,0.239437,41.288599,4.284559,22.988480,0.126214,88.380874,5.971081,52.742994,1.052597,943.254664,...,41.221707,1.679109e+03,0.432896,1.463455e+04,80.748748,8.114382e+03,-0.131599,2.380331e+04,114.222973,1.075644e+04
tether_usdtOpen,-0.000084,0.000013,0.001800,0.000009,-0.000047,0.000006,0.000930,0.000005,-0.000032,0.000004,...,0.000438,1.293485e-07,0.000014,1.001100e-05,0.002061,5.764941e-06,0.000039,1.175339e-05,0.002531,5.348701e-06
xrpOpen,0.000084,0.000063,0.005035,0.000038,0.000042,0.000109,0.006588,0.000066,-0.000004,0.000216,...,0.016421,3.396950e-04,0.000074,3.261033e-03,0.031898,2.243533e-03,-0.000821,5.203024e-03,0.043919,3.274797e-03


In [ ]:
autocorrelation

In [ ]:
approximate_entropy

In [ ]:


# Step 4: Interpretation of Extracted Features
# Analyze the extracted features and their relevance to the datasetset.
# Understand how these features can be used to gain insights.
# We will be discussing these in the pdf


# Step 5: Interactive Visualizations
# Create interactive visualizations for easy comparative analysis using both matplotlib and plotly.

# Using matplotlib:
# 1. Line Plot from matplotlib
#
plt.figure(figsize=(10, 6))

for dataset in datasets_extracted:
  for column in dataset.columns:
      plt.plot(dataset[column], label=column)
      plt.xlabel("Date")
      plt.ylabel("Value")
      plt.title("Time Series dataset")
      plt.legend()
      plt.show()



In [ ]:
#sns/matplotlib = 2nd vizualization tool

import seaborn as sns
import matplotlib.pyplot as plt



for dataset in datasets_extracted:
  for column in dataset.columns:
    # Set Seaborn style using a built-in theme (e.g., "darkgrid")
    sns.set_theme(style="darkgrid")

    # Assuming 'data' is your DataFrame with a timestamp index and a target column

    # Create a new figure with specified size
    plt.figure(figsize=(10, 6))

    # Plotting the time series using Seaborn lineplot with more customization

    sns.lineplot(data=dataset, x=dataset.index, y=column, color='skyblue', linewidth=2.5, marker='o', markersize=8)

    # Adding labels and title with custom font size
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Value', fontsize=16)
    plt.title(f'Time Series of {column}',fontsize=14)

    # Adding a grid and adjusting the style of grid lines
    plt.grid(True, linestyle='--', alpha=0.7)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Customize y-axis ticks with thousands separator and font size

    # Adding a legend with custom location
    plt.legend([f'Target Column: {column}'], loc='upper left')

    # Ensure tight layout for better spacing
    plt.tight_layout()

    # Display the plot
    plt.show()

In [ ]:
# Using plotly: #plotly = 3rd vizualization tool
# 3. Interactive Line Plot

fig = px.line(data_frame=approximate_entropy, x=approximate_entropy.index, y=approximate_entropy.columns, title="Interactive Time Series Data")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Value")
fig.show()



In [ ]:

fig = px.line(data_frame=autocorrelation, x=autocorrelation.index, y=autocorrelation.columns, title="Interactive Time Series Data")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Value")
fig.show()